<h2> Homework 1 Part B </h2>
<i> Gawan Dekker 11025654, Marvin Lau 12364282, Bobbie van Gorp 11161108</i>

<h3> Implementation </h3>

In [1]:
#Imports
import numpy as np

In [2]:
#Generate input

def gen_input_unsorted(length, n):
    out = []
    out.append([0] * length)
    for i in range(length):
        for j in range(1, n):
            affix = [0] * i + [j]
            suffixes = gen_input_unsorted(length - (i + 1), n)
            for suffix in suffixes:
                out.append(affix + suffix)
    return out

def gen_input(length, n):
    if length > 0:
        out = gen_input_unsorted(length, n)
        out.sort()
    else:
        out = []
    return out

def gen_input_pairs(length, n):
    out = gen_input(length * 2, n)
    for i in range(len(out)):
        out[i] = (out[i][:length], out[i][length:])
    return out


def get_conflicts(n, length, _in=[], ordered=False):
    if (len(_in) < length and n <= length):
        out = []
        # Add all possible conflicts.
        for i in [x for x in range(1, n + 1) if x not in _in]:
            out += get_conflicts(n, length, _in + [i], ordered)
            if ordered:
                break
        # Add absence of a conflict if possible.
        if (length - len(_in) > n - sum([1 for i in _in if i != 0])):
            out += get_conflicts(n, length, _in + [0], ordered)
        return out
    else:
        return [_in]

def add_conflicts(pair):
    out = []
    # Add all possible id conflicts to input pair.
    for n in range(len(pair[0]) + 1):
        for ids0 in get_conflicts(n, len(pair[0]), ordered=True):
            ranking0 = list(zip(pair[0], ids0))
            for ids1 in get_conflicts(n, len(pair[0])):
                out.append((ranking0, list(zip(pair[1], ids1))))
    # Remove any pairs in which conflicts appear
    # where relevance grades do not match.
    for i in range(len(out) - 1, -1, -1):
        delete = False
        for r0, id0 in out[i][0]:
            if id0 > 0:
                for r1, id1 in out[i][1]:
                    if id0 == id1 and not r0 == r1:
                        delete = True
                        break
            if delete:
                out.pop(i)
                break
    return out


def ERR(g_list, R_func=lambda g, max_g: float(2**g- 1) / 2**max_g):
    p = 1
    ERR = 0
    max_g = max(g_list)
    for r in range(1, len(g_list) + 1):
        R = R_func(g_list[r - 1], max_g)
        ERR += p * R / float(r)
        p *= 1 - R
    return ERR

In [3]:
#Interleaving

def td_interleaving(ranking_pair,max_interleav=3):
    ranking_p = ranking_pair[0] #[(0,0),(0,0),(0,0)] form or duplicate [(0,1),(0,0),(0,0)]
    ranking_e = ranking_pair[1]
    interleaved = []

    p_team = 0 #Amount results assigned from p
    e_team = 0
    p_pointer = 0 #Next top result from ranking p
    e_pointer = 0
    found_duplicates = [] #Duplicate documents have an ID of greater than 0. A matching number is an duplciate
    limit = len(ranking_p)

    while len(interleaved) < max_interleav:

        p_priority = np.random.choice(2, 1)[0]
        new_result = False
        if (p_team < e_team) or (p_team == e_team and p_priority == 1):

            while not new_result:
                top_result_p = ranking_p[p_pointer]
                relevance_p, duplicate_id_p = top_result_p

                p_pointer += 1
                if duplicate_id_p not in found_duplicates:
                    new_result = True
                    break
                elif p_pointer == limit:
                    break

            if new_result:
                interleaved.append((relevance_p, 0))
                p_team += 1

                if duplicate_id_p > 0:
                    found_duplicates.append(duplicate_id_p)
        else:

            while not new_result:
                top_result_e = ranking_e[e_pointer]
                relevance_e, duplicate_id_e = top_result_e
                e_pointer += 1

                if duplicate_id_e not in found_duplicates:
                    new_result = True
                    break
                elif e_pointer == limit:
                    break

            if new_result:
                interleaved.append((relevance_e, 1))
                e_team += 1

                if duplicate_id_e > 0:
                    found_duplicates.append(duplicate_id_e)

    return interleaved

def get_softmax(ranking_indices,tau=3):
    """
    
    """

    numerator_list = [] #Numerator values for each of the ranked results
    softmax_distribution = []

    for rank_index in ranking_indices:
        rank = rank_index + 1
        numerator_value = 1/(rank**tau)
        numerator_list.append(numerator_value)

    denominator = sum(numerator_list)

    for value in numerator_list:
        probability = value/denominator
        softmax_distribution.append(probability)

    return softmax_distribution

def prob_interleaving(ranking_pair,max_interleav=3):
    """
    """

    ranking_p = ranking_pair[0] #[(0,0),(0,0),(0,0)] form or duplicate [(0,1),(0,0),(0,0)]
    ranking_e = ranking_pair[1] #[(0,0),(0,0),(0,0)] form or duplicate #[(0,1),(0,0),(0,0)]
    interleaved = []
    limit = len(ranking_p)

    p_indices = list(range(limit))
    e_indices = list(range(limit))

    found_duplicates = []

    while len(interleaved) < max_interleav:

        p_priority = np.random.choice(2, 1)[0]

        if (p_priority and len(p_indices) > 0) or len(e_indices) == 0:
            softmax_p = get_softmax(p_indices)
            doc_index_p = np.random.choice(p_indices, 1, p=softmax_p)[0]
            p_indices.remove(doc_index_p)

            result_p = ranking_p[doc_index_p]
            relevance_p, duplicate_id_p = result_p

            if duplicate_id_p == 0 :
                interleaved.append((relevance_p, 0))
            elif (duplicate_id_p > 0 and duplicate_id_p not in found_duplicates):
                interleaved.append((relevance_p, 0))
                found_duplicates.append(duplicate_id_p)
                duplicate_index = ranking_e.index(result_p)
                e_indices.remove(duplicate_index)
        else:
            softmax_e = get_softmax(e_indices)
            doc_index_e = np.random.choice(e_indices, 1, p=softmax_e)[0]
            e_indices.remove(doc_index_e)

            result_e = ranking_e[doc_index_e]
            relevance_e, duplicate_id_e = result_e

            if duplicate_id_e == 0:
                interleaved.append((relevance_e, 1))
            elif (duplicate_id_e > 0 and duplicate_id_e not in found_duplicates):
                interleaved.append((relevance_e, 1))
                found_duplicates.append(duplicate_id_e)
                duplicate_index = ranking_p.index(result_e)
                p_indices.remove(duplicate_index)

    return interleaved

<h3> Documentation </h3>

<h3> Generate Input </h3>
<ul>
<li>gen_input_unsorted(length, n) <br>
    Creates a list of all possible combinations of relevance scores.
    
    Parameters
    ----------
    length : int
        Length of a combination of relevance scores.
    n : int
        Maximum relevance score
    
    Returns
    -------
    out : list
        A list containing all possible combinations of relevance scores.
    

</li>

<li>gen_input(length, n) <br>
    Creates a sorted list of all possible combinations
    of relevance scores.
    
    Parameters
    ----------
    length : int
        Length of a combination of relevance scores.
    n : int
        Maximum relevance score
    
    Returns
    -------
    out : list
        A sorted list containing all possible combinations
        of relevance scores.

    
</li>

<li>     gen_input_pairs(length, n) <br>
    Creates a sorted list of all possible pairs of combinations
    of relevance scores.
    
    Parameters
    ----------
    length : int
        Length of a combination of relevance scores.
    n : int
        Maximum relevance score
    
    Returns
    -------
    out : list
        A sorted list containing all possible pairs of combinations
        of relevance scores.
</li>

<li>get_conflicts(n, length, _in=[], ordered=False) <br>
    Creates a list of all possible id conflicts.
    
    Parameters
    ----------
    n : int
        Total possible number of id conflicts at a time.
    length: int
        Length of the list containing id conflicts.
    _in : array_like
        Array of current id conflicts.
    ordered : bool
        Indicates whether id conflict numbers are to appear in order
        or are permitted to appear in any order.
    
    Returns
    -------
    out : list
        A list containing all possible id conflicts.
        A value of 0 indicates no conflict,
        a value higher than indicates a conflict.

    
</li>

<li>add_conflicts(pair) <br>
    Adds id conflicts to a tuple of combinations of relevance grades.
    
    Parameters
    ----------
    pair : tuple
        Tuple of combinations of relevance grades.
    
    Returns
    -------
    out : list
        A list of all possible tuples of combinations
        of relevance grades with id conflicts.
        Relevance grades from the inputs have been replaced by a tuple
        of relevance grade and id conflict number.
    
    
</li>

<li>ERR(g_list, R_func=lambda g, max_g: float(2**g- 1) / 2**max_g) <br>
    Calculates Expected Reciprocal Rank for one list
    of relevance grades.
    
    Source
    ------
    The algorithm originates from a paper by O. Chapelle et al.
    named "Expected Reciprocal Rank for Graded Relevance"
    and can be found here: http://olivier.chapelle.cc/pub/err.pdf
    
    Parameters
    ----------
    g_list : array_like
        Array of relevance grades.
    R_func : function(int, int) -> float
        Function that converts a relevance grade
        to probability of relevance.
    
    Returns
    -------
    ERR : float
        Expected Reciprocal Rank.
    
    
</li>

</ul>

<h3> Interleaving </h3>
<ul>
<li>td_interleaving(ranking_par,max_interleav=3) <br>
Run Team-draft interleaving given a ranking pair as input

    Parameters
    ----------
    An ranking pair : List of ranked and labeled results
        Index of list elements represents the rank of that element. Each
        element is a tuple of the relevance score and a duplicate ID.
        If this value is 0, it has no duplicate with a document in the results of other ranker.
        If the value is greater than 0, then it has a duplicate with another result of the other ranker that matches this number
        No duplicate example: E ranked list:  [(0,0),(0,0),(0,0)] and P ranked list: [(1,0),(0,0),(0,0)] form
        2 duplicates example: E ranked list: [(1,1),(0,2),(0,0)]  and P ranked list: [(1,0),(1,1),(0,2)]  for example
    
    Returns
    -------
    Interleaved list (of length 3 as default) based on Team-draft method: list
        Index + 1 represents the rank of the interleaved list and element is an tuple of the form (relevance: binary,ranker credit:binary), Credits are assigned as P(0) and E(1)
</li>        
        
<li> get_softmax(ranking_indices,tau=3)<br>
    Compute softmax distribution for a ranker given the indices of documents that are avaliable to be picked.

    Parameters
    ----------
    Ranking indices : List of indices that represent the rank of documents that can be picked for next interleaving.
    Example of nothing picked of ranker: List of the form: [0,1,2]
    Example of document already picked of ranker: List of the form: [0,2]

    Returns
    -------
    List of softmax probabilities for documents to be picked: List
        The list maps one to one the ranking indices that were given as input, each element is a probability for the document for that particular index
        in that ranking indices list, which in turn consist of numbers that represents the actual indices to be picked from ranked results.
</li>

<li> prob_interleaving(ranking_pair,max_interleav=3)<br>
    Run Probabilistic interleaving  given a ranking pair as input

    Parameters
    ----------
    An ranking pair : List of ranked and labeled results
        Index of list elements represents the rank of that element. Each
        element is a tuple of the relevance score and a duplicate ID.
        If this value is 0, it has no duplicate with a document in the results of other ranker.
        If the value is greater than 0, then it has a duplicate with another result of the other ranker that matches this number
        No duplicate example: E ranked list:  [(0,0),(0,0),(0,0)] and P ranked list: [(1,0),(0,0),(0,0)] form
        2 duplicates example: E ranked list: [(1,1),(0,2),(0,0)]  and P ranked list: [(1,0),(1,1),(0,2)]  for example
    Returns
    -------
    Interleaved list (of length 3 as default) based on probabilistc interleaving method: list
        Index + 1 represents the rank of the interleaved list and element is an tuple of the form (relevance: binary,ranker credit:binary), credits are assigned as P(0) and E(1)
    
    
</li>
</ul>

<h3> Analysis </h3>